# 뉴스 요약봇 만들기
-----
### 프로젝트 - 뉴스기사 요약해보기

## <span style="color:purple">순서</span>
> **Step1.** 인물모드 직접 해 보기<br>
**Step2.** 사진에서 문제점 찾기<br>
**Step3.** 해결 방법을 제안해 보기


### <span style="color:green">Step1. 데이터 수집하기</span>
---
데이터는 아래 링크에 있는 뉴스 기사 데이터(newssummarymore.csv)를 사용하세요.
sunnysai12345/News_Summary

아래의 코드로 데이터를 다운로드 할 수 있어요.

In [1]:
import nltk
nltk.download('stopwords')

import numpy as np
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from bs4 import BeautifulSoup 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package stopwords to /home/aiffel-
[nltk_data]     dj49/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import urllib.request

urllib.request.urlretrieve("https://raw.githubusercontent.com/sunnysai12345/News_Summary/master/news_summary_more.csv", filename="news_summary_more.csv")
data = pd.read_csv('news_summary_more.csv', encoding='iso-8859-1')

In [3]:
data.sample(10)

,headlines,text
93729,For each slap on my Jawan lay down 100 jihadi ...,Gautam Gambhir posted a series of tweets in re...
57003,Badly need work as my mom is in ICU: 'Kyunki S...,"Television actress Jaya Bhattacharya, known fo..."
48304,"Kohli surpasses Azhar, Gayle in highest ODI ru...",Following his 75-run knock against South Afric...
11563,What are some facts about world's tallest stat...,The 182-metre-tall Statue of Unity honouring S...
63083,"Sweden plans to build 50,000 new nuclear bunkers",The Swedish government is planning to build 50...
90646,"India summons Pak envoy, seeks action on soldi...",India on Wednesday summoned Pakistan High Comm...
76537,HC accepts Maha govt's 14 yrs age restriction ...,The Bombay High Court on Monday accepted Mahar...
84473,Pak team performs 'Sajda' after Champions Trop...,The entire Pakistani cricket team performed 'S...
32735,Dalmia Group adopts Gandikota Fort along with ...,The Dalmia Bharat Group has adopted Gandikota ...
75055,Man flies to Amazon headquarters after account...,An Utah-based Amazon seller flew to the compan...


이 데이터는 기사의 본문에 해당되는 text와 headlines 두 가지 열로 구성되어져 있습니다.

추상적 요약을 하는 경우에는 text를 본문, headlines를 이미 요약된 데이터로 삼아서 모델을 학습할 수 있어요. 추출적 요약을 하는 경우에는 오직 text열만을 사용하세요.

In [4]:
data = pd.read_csv(os.getenv("HOME")+"/aiffel/news_summarization/data/Reviews.csv", nrows = 100000)

print('전체 샘플수 :',(len(data)))

전체 샘플수 : 100000


In [5]:
data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [6]:
data = data[['Text','Summary']]
data.head()

#랜덤한 15개 샘플 출력
data.sample(15)

,Text,Summary
84025,These are by far the best-tasting saltines on ...,The BEST Saltines
80509,"This stuff is so tasty, especially the caramel...",Yummy!
71532,"We couldn't decide which we loved more, pomeg...",Blueberry covered in dark Chocolate
84089,"I've submitted this review once before, and it...","Great food, but totally misleading weight"
18762,If you are looking for a product that is going...,Weight Loss vs. Energy Drink
11252,"I don't even like coffee, but this is in a cla...","I don't even like coffee, but I LOVE this!"
15864,Was pleased with the packaging and prompt deli...,Order arrived promptly filled with EMPTY k-cups
5775,I bought the vanilla beans to make DIY vanilla...,no imitation anymore
16332,"Love this coffee, it has just the right amount...",Green Mountain coffee breakfast blend.
72122,"Yes, there's NO sodium and it still tastes gre...",Makes terrific chicken with no sodium


**Error]** **<span style="color:Orange">에러명</span>**: 사유

### <span style="color:green">Step2. 데이터 전처리하기 (추상적 요약)</span>
---
실습에서 사용된 전처리를 참고하여 각자 필요하다고 생각하는 전처리를 추가 사용하여 텍스트를 정규화 또는 정제해 보세요. 만약, 불용어 제거를 선택한다면 상대적으로 길이가 짧은 요약 데이터에 대해서도 불용어를 제거하는 것이 좋을지 고민해보세요.

In [7]:
print('Text 열에서 중복을 배제한 유일한 샘플의 수 :', data['Text'].nunique())
print('Summary 열에서 중복을 배제한 유일한 샘플의 수 :', data['Summary'].nunique())

Text 열에서 중복을 배제한 유일한 샘플의 수 : 88426
Summary 열에서 중복을 배제한 유일한 샘플의 수 : 72348


In [8]:
data.drop_duplicates(subset = ['Text'], inplace = True)

print('전체 샘플수 :',(len(data)))

전체 샘플수 : 88426


In [9]:
print(data.isnull().sum())

Text       0
Summary    1
dtype: int64


In [10]:
data.dropna(axis = 0, inplace = True)
print('전체 샘플수 :',(len(data)))

전체 샘플수 : 88425


### <span style="color:green">Step3. 어텐션 메커니즘 사용하기 (추상적 요약)</span>
---
일반적인 seq2seq보다는 어텐션 메커니즘을 사용한 seq2seq를 사용하는 것이 더 나은 성능을 얻을 수 있어요. 실습 내용을 참고하여 어텐션 메커니즘을 사용한 seq2seq를 설계해 보세요.

### <span style="color:green">Step4. 실제 결과와 요약문 비교하기 (추상적 요약)</span>
---
실습에서 사용된 전처리를 참고하여 각자 필요하다고 생각하는 전처리를 추가 사용하여 텍스트를 정규화 또는 정제해 보세요. 만약, 불용어 제거를 선택한다면 상대적으로 길이가 짧은 요약 데이터에 대해서도 불용어를 제거하는 것이 좋을지 고민해보세요.

### <span style="color:green">Step5. Summa을 이용해서 추출적 요약해보기</span>
---
추상적 요약은 추출적 요약과는 달리 문장의 표현력을 다양하게 가져갈 수 있지만, 추출적 요약에 비해서 난이도가 높아요. 반대로 말하면 추출적 요약은 추상적 요약에 비해 난이도가 낮고 기존 문장에서 문장을 꺼내오는 것이므로 잘못된 요약이 나올 가능성이 낮아요.

Summa의 summarize를 사용하여 추출적 요약을 해보세요.

# <span style="color:purple">루브릭 평가 기준</span>
---
번호|평가문항|상세기준
---|---|---
1 |Abstractive 모델 구성을 위한 텍스트 전처리 단계가 체계적으로 진행되었다.|분석단계, 정제단계, 정규화와 불용어 제거, 데이터셋 분리, 인코딩 과정이 빠짐없이 체계적으로 진행되었다.
2 |텍스트 요약모델이 성공적으로 학습되었음을 확인하였다.|모델학습이 안정적으로 수렴되었음을 그래프를 통해 확인하였으며, 실제 요약문과 유사한 요약문장을 얻을 수 있었다.
3 |Extractive 요약을 시도해 보고 Abstractive 요약 결과과 함께 비교해 보았다.|두 요약 결과를 문법완성도 측면과 핵심단어 포함 측면으로 나누어 비교분석 결과를 제시하였다./center>

# <span style="color:purple">회고</span>
---